In [ ]:
# Scrape Meta Data For All Solar Arrays
### Url: http://dkasolarcentre.com.au/

## Scrape Array Meta Data

In [214]:
root_site = "http://dkasolarcentre.com.au/historical-data/download"
root_data = requests.get(root_site).text
root_soup = BeautifulSoup(root_data)

link_list = root_soup.find(class_='download-list').find_all('a')

array_number = root_soup.find_all("span", class_="source-number-label")
array_number = [x.text for x in array_number]
array_number = array_number[0:40]

link_list = [x.get('href') for x in link_list][1::2]
link_list = link_list[0:40]

arrays_links = list(zip(link_list,array_number))

In [222]:

def return_panel_data(array_url):
    
    panel_data = requests.get(array_url).text
    panel_soup = BeautifulSoup(panel_data)
    tables = panel_soup.find_all('tr')
    features = []
    values = []
    
    for table in tables:

        feature = table('th')[0].text
        feature = re.sub('\s+', ' ', feature).strip()
        features.append(feature)

        value = table('td')[0].text
        value  = re.sub('\s+', ' ', value).strip()
        values.append(value)

    return (list(zip(features,values)))

In [294]:

def build_panel_meta_database(array_links):
    
    panel_meta_data = pd.DataFrame()

    for array in arrays_links:
        url = array[0]
        panel_number = array[1]
        print('Getting: ' +str(panel_number))
        panel_features = return_panel_data(url)

        for feature in panel_features:
            panel_meta_data.loc[panel_number,feature[0]] = feature[1]
    return panel_meta_data


In [319]:
panel_meta_data = build_panel_meta_database(arrays_links)

Getting: 1A
Getting: 1B
Getting: 2
Getting: 3
Getting: 4
Getting: 5
Getting: 6
Getting: 7
Getting: 8
Getting: 9A
Getting: 10
Getting: 11
Getting: 12
Getting: 13
Getting: 14
Getting: 16A
Getting: 16B
Getting: 16C
Getting: 16D
Getting: 17
Getting: 18
Getting: 19
Getting: 20
Getting: 21
Getting: 22
Getting: 23
Getting: 24
Getting: 25
Getting: 26
Getting: 28
Getting: 29
Getting: 30
Getting: 31
Getting: 32
Getting: 33
Getting: 34
Getting: 35
Getting: 36
Getting: 37
Getting: 38


In [320]:
panel_meta_data.to_csv('panel_meta_data.csv')

# Scrape Fault Data

In [124]:
# Scrape Fault Data

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

fault_site = "http://dkasolarcentre.com.au/historical-data/notes-on-the-data"
fault_data = requests.get(fault_site).text
fault_soup = BeautifulSoup(fault_data)
fault_list=fault_soup.find("ol", class_="child-listing")
fault_listings =fault_list.find_all('li', recursive=False)

In [316]:

def return_fault_df(fault_listings):
    
    fault_data = pd.DataFrame(columns=(cols))

    i = 0
    for item in fault_listings:
        fault_ID = i
        date = item.find("span", class_='date').text
        date = re.sub('\s+', ' ', date).strip()
        description = item('p')[1].text
        title = item('h2')[0].text
        arrays = [x.text for x in item.find_all("span", class_="source-number-label")]

        fault_data.loc[fault_ID,'Date'] = date
        fault_data.loc[fault_ID,'Title'] = title
        fault_data.loc[fault_ID,'Description'] = description

        for array in arrays:
            fault_data.loc[fault_ID, array] = 1

        i += 1

    fault_data.index = pd.to_datetime(fault_data.Date)
    fault_data = fault_data.drop('Date', axis=1)
    
    return fault_data

In [317]:
fault_data = return_fault_df(fault_listings)

In [321]:
fault_data.to_csv('fault_data_07_12_19.csv')